In [ ]:
import warnings

warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
import tensorflow as tf
import dask.dataframe as dd
import matplotlib.pyplot as plt                                                                      
%matplotlib inline   

tf.random.set_seed(2)

train = dd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv').compute().sort_values('ts_id')
features = [c for c in train.columns if "feature" in c]


def create_mlp(
    num_columns, num_labels, hidden_units, dropout_rates, label_smoothing, learning_rate
):

    inp = tf.keras.layers.Input(shape=(num_columns,))
    x = tf.keras.layers.BatchNormalization()(inp)
    x = tf.keras.layers.Dropout(dropout_rates[0])(x)
    for i in range(len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(tf.keras.activations.swish)(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 1])(x)

    x = tf.keras.layers.Dense(num_labels)(x)
    out = tf.keras.layers.Activation("sigmoid")(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
        loss=tf.keras.losses.BinaryCrossentropy(label_smoothing=label_smoothing),
        metrics=tf.keras.metrics.AUC(name="AUC"),
    )

    return model


batch_size = 4096
hidden_units = [384, 896, 896, 394]
dropout_rates = [
    0.10143786981358652,
    0.15720339053599725,
    0.2203017847244654,
    0.23148340929571917,
    0.2457768967777311,
]
label_smoothing = 1e-2
learning_rate = 1e-5

num_models = 2

models = []
for i in range(num_models):
    clf = create_mlp(
        len(features), 1, hidden_units, dropout_rates, label_smoothing, learning_rate
    )
    clf.load_weights(f"../input/js-nn-models/JSModel_{i}.hdf5")
    models.append(clf)

f_mean = np.load('../input/js-nn-models/f_mean.npy')

In [ ]:
opt_th = 0.5034

In [ ]:
train['trend'] = train['resp'].cumsum()                                                              
train['weighted_trend'] = (train['weight']*train['resp']).cumsum()                                   
train.plot(x='ts_id', y=['trend', 'weighted_trend'])                                                 
plt.show()

In [ ]:
import janestreet

env = janestreet.make_env()
env_iter = env.iter_test()

for test_df, pred_df in env_iter:
    if test_df["weight"].item() > 0:
        x_tt = test_df.loc[:, features].values
        if np.isnan(x_tt[:, 1:].sum()):
            x_tt[:, 1:] = np.nan_to_num(x_tt[:, 1:]) + np.isnan(x_tt[:, 1:]) * f_mean
        pred = 0.0
        for clf in models:
            pred += clf(x_tt, training=False).numpy().item() / num_models
        pred_df.action = np.where(pred >= opt_th, 1, 0).astype(int)
    else:
        pred_df.action = 0
    env.predict(pred_df)